In [2]:
# 공통
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from dateutil.relativedelta import relativedelta
import datetime
import matplotlib.pyplot as plt

In [8]:
# Train 데이터 불러오기 (00년 ~ 21년)
# 2021년 Train / 2022년 2022Train

data = pd.read_csv('전체.csv', encoding='cp949')
# data = data[38:]

# data = data.drop(['price'], axis = 1)
data = data.drop(['전체_temp'], axis = 1)
data = data.drop(['전체_hum'], axis = 1)

data = data.drop(['강원_temp'], axis = 1)
data = data.drop(['경기_temp'], axis = 1)
data = data.drop(['경남_temp'], axis = 1)
data = data.drop(['경북_temp'], axis = 1)
data = data.drop(['광주_temp'], axis = 1)
data = data.drop(['대구_temp'], axis = 1)
data = data.drop(['대전_temp'], axis = 1)
data = data.drop(['부산_temp'], axis = 1)
data = data.drop(['서울_temp'], axis = 1)
data = data.drop(['세종_temp'], axis = 1)
data = data.drop(['울산_temp'], axis = 1)
data = data.drop(['충남_temp'], axis = 1)
data = data.drop(['충북_temp'], axis = 1)
data = data.drop(['전남_temp'], axis = 1)
data = data.drop(['전북_temp'], axis = 1)
data = data.drop(['제주_temp'], axis = 1)
data = data.drop(['인천_temp'], axis = 1)

data = data.drop(['강원_hum'], axis = 1)
data = data.drop(['경기_hum'], axis = 1)
data = data.drop(['경남_hum'], axis = 1)
data = data.drop(['경북_hum'], axis = 1)
data = data.drop(['광주_hum'], axis = 1)
data = data.drop(['대구_hum'], axis = 1)
data = data.drop(['대전_hum'], axis = 1)
data = data.drop(['부산_hum'], axis = 1)
data = data.drop(['서울_hum'], axis = 1)
data = data.drop(['세종_hum'], axis = 1)
data = data.drop(['울산_hum'], axis = 1)
data = data.drop(['충남_hum'], axis = 1)
data = data.drop(['충북_hum'], axis = 1)
data = data.drop(['전남_hum'], axis = 1)
data = data.drop(['전북_hum'], axis = 1)
data = data.drop(['제주_hum'], axis = 1)
data = data.drop(['인천_hum'], axis = 1)
    
data = data.set_index('date')
# data = data.fillna(method='ffill')
data.tail(10)

,price
date,
2021-12-22,48838343
2021-12-23,40257093
2021-12-24,44660225
2021-12-25,71313940
2021-12-26,67554989
2021-12-27,48002138
2021-12-28,38070210
2021-12-29,52093193
2021-12-30,47463341


In [5]:
# # 2021년 예측

# train = data[:'2021-11-30']['price']

# exog1 = data.drop('price', axis=1)[:'2021-11-30']
# exog1 = sm.add_constant(exog1)

# exog2 = data.drop('price', axis=1)['2021-12-01':]
# exog2 = sm.add_constant(exog2)

In [11]:
# 인덱스 지정(?)

# strptime은 날짜와 시간 형식의 문자열을 datetime으로 변환해 줌
# strftime은 날짜와 시간(datetime)을 문자열로 출력
start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
date_list = [start + relativedelta(days = x) for x in range(0, len(data)) ]

data['index'] = date_list

# set_index는 데이터프레임을 먼저 불러온 후, 특정 열을 인덱스로 설정할 때 사용
data.set_index( ['index'], inplace=True) 

# index 출력 X
data.index.name=None
data.head()

,price
2021-01-01,39209808
2021-01-02,62239672
2021-01-03,57365413
2021-01-04,95785399
2021-01-05,37415973


In [12]:
# coef값은 0에서 가장 떨어졌을때 가장 이상적
# p 값은 0과 가장 가까울때가 가장 좋습니다

mod = sm.tsa.statespace.SARIMAX(
    data['price'],
    order = (1,0,0),
    seasonal_order = (2,0,1,7),
    # AR 매개 변수를 변환하여 모델의 자동 회귀 구성 요소에서 정상 성을 강제 적용할지 여부입니다. 기본값은 True입니다.
    # AR항이 stationary를 만족하게끔 강제하는 것
    enforce_stationarity = False,
    # 모델의 이동 평균 구성 요소에서 반전 성을 강제하기 위해 MA 매개 변수를 변환할지 여부입니다. 기본값은 True입니다.
    # MA항이 starionary를 만족하게끔 강제하는 것
    enforce_invertibility = False
)

results = mod.fit()

# print(results.summary())
print( results.summary().tables[1] )

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.2822      0.081      3.469      0.001       0.123       0.442
ar.S.L7        0.9253      0.098      9.432      0.000       0.733       1.118
ar.S.L14       0.0739      0.097      0.764      0.445      -0.116       0.263
ma.S.L7       -0.7110      0.071    -10.014      0.000      -0.850      -0.572
sigma2      5.561e+14        nan        nan        nan         nan         nan


In [13]:
# 2022년 예측

train = data[:'2021-12-31']['price']

exog1 = data.drop('price', axis=1)[:'2021-12-31']

exog2 = data.drop('price', axis=1)['2022-01-01':]

In [105]:
# start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# # 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
# df_list = [start + relativedelta(days = x) for x in range(0, len(data)) ]

# # 그리드 서치(모델에게 가장 적합한 하이퍼 파라미터를 찾기) p, q

# import itertools

# # Define the p, d and q parameters to take any value between 0 and 2
# p = d = q = range(0, 2)

# # Generate all different combinations of p, q and d triplets
# pdq = list( itertools.product(p, d, q))

# # Generate all different combinations of seasonal p, q and d triplets
# seasonal_pdq = [ (x[0], x[1], x[2], 7)  for x in pdq ]

# print('Example of parameter combinations for Seasonal ARIMA ...')
# print('SARIMAX:  {} x {}'.format(pdq[1], seasonal_pdq[1]) )
# print('SARIMAX:  {} x {}'.format(pdq[1], seasonal_pdq[2]) )
# print('SARIMAX:  {} x {}'.format(pdq[2], seasonal_pdq[3]) )
# print('SARIMAX:  {} x {}'.format(pdq[2], seasonal_pdq[4]) )

In [106]:
# # AIC(통계 모델을 성능을 측정하는 기준) - 낮을수록 좋음

# select_candi = 10000000
# param_candi = ( 0, 0, 0 )
# param_seasonal_candi = ( 0, 0, 0)

# count=0
# end_count = len(pdq)

# for param in pdq:   
#     for param_seasonal in seasonal_pdq:
#         try:
#             mod = sm.tsa.statespace.SARIMAX(data['price'],
#                                             order=param,
#                                             seasonal_order=param_seasonal,
#                                             enforce_stationarity=False,
#                                             enforce_invertibility=False
#                                            )
#             results = mod.fit()
#             count += 1
#             if count <= 5:
#                 print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
            
#             if results.aic < select_candi:
#                 select_candi = results.aic
#                 param_candi = param
#                 param_seasonal_candi = param_seasonal
#         except:
#             continue
            
# print(param_candi, param_seasonal_candi, select_candi) 

In [107]:
# def predict(exog1, exog2):

#     mod = sm.tsa.statespace.SARIMAX(train, exog = exog1, order = (1, 0, 1), seasonal_order = (0, 1, 1, 7))
#     fit_res = mod.fit(disp = False)

#     forecast = fit_res.forecast(steps = 31, exog = exog2)

#     # 2021년 실제값
#     actual = data['price'][-31:]

#     # # 2022년 실제값
#     # actual = [2619529000, 2480632000, 2778454000, 2637731000, 2831080000]

#     rmse = np.sqrt(mean_squared_error(actual, forecast))
    
#     return rmse

# # order = (1,0,0), seasonal_order = (2,0,1,7)
# # order = (1, 0, 1), seasonal_order = (0, 1, 1, 7)

In [108]:
# predict(exog1, exog2)

# order = (1,0,0), seasonal_order = (2,0,1,7) / order = (1, 0, 1), seasonal_order = (0, 1, 1, 7)

# 새로운 arima 코드.ipynb / 찬재.ipynb

# 변수없이 12032081.903667446 / 12822899.653348675
# 전체평균 11134711.02414859 / 14996102.978741448
# 전체평균제외 13945093.033749819 / 15015795.613447221
# 온도만 13141867.131440545 / 13840825.71410531
# 습도만 12218263.756765371 / 13670886.534086179

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


13670886.534086179

In [14]:
mod = sm.tsa.statespace.SARIMAX(train, exog = exog1, order = (1, 0, 0), seasonal_order = (2, 0, 1, 7))
fit_res = mod.fit(disp = False)

forecast = fit_res.forecast(steps = 365, exog = exog2)

forecast

ValueError: zero-size array to reduction operation maximum which has no identity

In [110]:
forecast.to_csv('전체평균.csv', index=False)

PermissionError: [Errno 13] Permission denied: '전체평균.csv'